# Getting started

This small draft contains the main steps for recreating data from [Capillary pressure and relative permeability estimation for low salinity waterflooding processes using pore network models](https://doi.org/10.1016/j.petrol.2019.106253). Please note OpenPNM V1.8 has been used for this demo.

## Case 1: LWSF in a sandstone sample

In [ ]:
import OpenPNM
import OpenPNM.Utilities.IO as io
import OpenPNM.Geometry.models as gm 
import stats_analysis as stta
import numpy as np
import pandas as pd 
import h5py 
from __MartinezMendoza2019__ import MartinezMendoza2019

### Network

In [ ]:
mgr = OpenPNM.Base.Workspace()
pn = OpenPNM.Network.GenericNetwork(name='pn')
prefix = 'S1'
io.Statoil.load(network=pn, path='', prefix=prefix)
pn.trim(pn.check_network_health()['trim_pores'])

### Build Geometry

In [ ]:
# Set up geometries only to internal pores

geometry = OpenPNM.Geometry.GenericGeometry(network=pn,pores=pn.Ps,throats=pn.Ts,name='geom')

for item in pn.props():
    if item not in ['throat.conns', 'pore.coords']:
        geometry.update({item: pn.pop(item)})

for item in pn.props():
    if item not in ['throat.conns', 'pore.coords','pore.all','throat.all',
                    'pore.outlets','pore.inlets','pore.coordination_number']:
        del(pn[item])
        
geometry['pore.diameter']= geometry['pore.radius']*2
geometry.add_model(propname='pore.volume',model=gm.pore_volume.sphere)
geometry.add_model(propname='pore.area',model=gm.pore_area.spherical)

geometry['throat.diameter']= geometry['throat.radius']*2
geometry.add_model(propname='throat.length',model=gm.throat_length.straight)
geometry.add_model(propname='throat.volume',model=gm.throat_volume.cylinder)
geometry.add_model(propname='throat.perimeter',model=gm.throat_perimeter.cylinder)

geometry.add_model(propname='throat.area',model=gm.throat_area.cylinder)
geometry.add_model(propname='throat.surface_area',model=gm.throat_surface_area.cylinder)

geometry.regenerate()

### Build phases

In [ ]:
'''oil'''
oil = OpenPNM.Phases.GenericPhase(network=pn,name='oil')
oil['pore.contact_angle'] = 180-111
oil['pore.temperature'] = 363.15  # K (90°C) 
oil['pore.viscosity'] = 0.010824  #Pa.s

'''water: formation water'''
water = OpenPNM.Phases.GenericPhase(network=pn,name='water')
water['pore.temperature'] = 363.15  # K (90°C) 
water['pore.viscosity'] = 3e-4  #Pa.s
water['pore.surface_tension']= 0.02343  #N.m
water['pore.diffusivity'] = 2.151e-9  # m2/s Qiao(2016)

'''sw: formation water/100'''
sw = OpenPNM.Phases.GenericPhase(network=pn,name='sw')
sw['pore.temperature'] = 363.15  # K (90°C) 
sw['pore.viscosity'] = 4.85e-4  #Pa.s
sw['pore.surface_tension']= 0.01724  #N.m
sw['pore.diffusivity'] = 2.151e-9  # m2/s Qiao(2016)

### Build physics

In [ ]:
phys_oil = OpenPNM.Physics.Standard(network=pn,phase=oil,pores=pn.Ps,
                                    throats=pn.Ts,name='phys_oil')   
phys_water = OpenPNM.Physics.Standard(network=pn,phase=water,pores=pn.Ps,
                                    throats=pn.Ts,name='phys_water') 
phys_sw = OpenPNM.Physics.Standard(network=pn,phase=sw,pores=pn.Ps,
                                   throats=pn.Ts,name='phys_sw')      
               

mod = OpenPNM.Physics.models.diffusive_conductance.bulk_diffusion
phys_water.add_model(propname='throat.diffusive_conductance',model=mod,
                  diffusivity='pore.diffusivity')
phys_water.regenerate()
phys_sw.add_model(propname='throat.diffusive_conductance',model=mod,
                  diffusivity='pore.diffusivity')
phys_sw.regenerate()


### Save network

In [ ]:
mgr.save_workspace('S1.pnm')

### Flow and permeability

A previous saved network is loaded using OpenPNM

In [ ]:
mgr = OpenPNM.Base.Controller()
mgr.clear()
mgr.load_workspace(filename='../S1.pnm') 

pn = mgr['pn']
geom = mgr['geom']

oil = mgr['oil']
water = mgr['water']
sw = mgr['sw']

Boundary conditions are set, and flow simulation is computed

In [ ]:
inlet = pn.pores('inlets')
outlet = pn.pores('outlets')
Pin = 2
Pout = 1

SF_water = OpenPNM.Algorithms.StokesFlow(network=pn,phase=water,name='SF_water')
SF_water.set_boundary_conditions(bctype='Dirichlet', bcvalue=Pin, pores=inlet)
SF_water.set_boundary_conditions(bctype='Dirichlet', bcvalue=Pout, pores=outlet)
SF_water.run()
SF_water.return_results()
Kwater = SF_water.calc_eff_permeability()
KwaterDarcy = Kwater  / 9.86923e-16
print('Absolute permeablity: %.2f [mD]' % (KwaterDarcy))

SF_sw = OpenPNM.Algorithms.StokesFlow(network=pn,phase=sw,name='SF_sw')
SF_sw.set_boundary_conditions(bctype='Dirichlet', bcvalue=Pin, pores=inlet)
SF_sw.set_boundary_conditions(bctype='Dirichlet', bcvalue=Pout, pores=outlet)
SF_sw.run()
SF_sw.return_results()
Ksw = SF_sw.calc_eff_permeability()
KswDarcy = Ksw  / 9.86923e-16

### Transient transport

Two brines are considered for sequentially injection: formation water invades the network first, which corresponds to the high salinity concentration (HS). In case 1, low salinity fluid (LS) is the formation water diluted 100 times.

The number of iterations *sims* and time step *Dt* are specified for performing transient transport

In [ ]:
sims = 7000
Dt = 1
outs = 70

**HS fluid injection**

Transport model is loaded, and boundary conditions are set

In [ ]:
print('Stage 0 has started')
stage0 = MartinezMendoza2019(network=pn, phase=water)
stage0.set_boundary_conditions(bctype='Dirichlet', bcvalue=216000, pores=inlet)
stage0.set_boundary_conditions(bctype='Neumann', bcvalue=0, pores=outlet)

After running transport algorithm, concentration results are stored in H5 format (HDF - Hierarchical Data Format)

In [ ]:
C_stage0 = stage0.run(c0=0, Dt=Dt, sims=sims, outputs=outs)

hf = h5py.File('S1-stage0-connate.h5', 'w')
hf.create_dataset('S1-stage0-connate', data=C_stage0)
hf.close()

If needed, the exported concentration data can be loaded for furhter analysys. For this demo, the h5 is imported back, and the concentration values are read and set to pores and throats.

**LS fluid injection**

The previous workflow is used for LS fluid injection

In [ ]:
hf = h5py.File('S1-stage0-connate.h5','r')
conc = hf.get('S1-stage0-connate')[:,-1]
conc = np.array(conc)

pn['pore.conc_HS'] = conc[pn.pores('all')]
pn['throat.conc_HS'] = conc[pn.Np+pn.throats('all')]
hf.close()

'''LS fluid injection'''
print('Stage 1 has started')
stage1 = MartinezMendoza2019(network=pn,phase=sw)
stage1.set_boundary_conditions(bctype='Dirichlet', bcvalue=2160, pores=inlet)
stage1.set_boundary_conditions(bctype='Neumann', bcvalue=0, pores=outlet)
C_stage1 = stage1.run(c0=C_stage0[:,-1], Dt=Dt, sims=sims, outputs=outs)
hf = h5py.File('S1-stage1-sw.h5', 'w')
hf.create_dataset('S1-stage1-sw', data=C_stage1)
hf.close()

hf = h5py.File('S1-stage1-sw.h5','r')
conc = hf.get('S1-stage1-sw')[:,-1]
conc = np.array(conc)